<a href="https://colab.research.google.com/github/natanrajch/DiploDatos/blob/main/aprendizaje_refuerzos/TicTacToe_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalaciones

In [3]:
!pip install stable-baselines3[extra,tests,docs]

     |████████████████████████████████| 174 kB 8.5 MB/s 
     |████████████████████████████████| 103 kB 54.6 MB/s 
     |████████████████████████████████| 64 kB 3.7 MB/s 
     |████████████████████████████████| 150 kB 56.7 MB/s 
     |████████████████████████████████| 2.6 MB 60.1 MB/s 
     |████████████████████████████████| 2.8 MB 81.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 69 kB 10.7 MB/s 
     |████████████████████████████████| 743 kB 49.9 MB/s 
     |████████████████████████████████| 749 kB 65.4 MB/s 
     |████████████████████████████████| 280 kB 81.5 MB/s 
     |████████████████████████████████| 213 kB 66.5 MB/s 
     |████████████████████████████████| 280 kB 84.7 MB/s 
     |████████████████████████████████| 280 kB 90.1 MB/s 
     |████████████████████████████████| 280 kB 83.3 MB/s 
     |████████████████████████████████| 279 kB 94.3 MB/s 
     |████████████████████████████████| 279 kB 66.2 MB/s 
     |██████████████

In [4]:
#@title Instalación de RLBaselinesZoo (no modificar)

# Estamos en Colab?

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    !cd rl-baselines3-zoo/
    !apt-get install swig cmake ffmpeg
    !pip install -r /content/rl-baselines3-zoo/requirements.txt

Cloning into 'rl-baselines3-zoo'...
remote: Enumerating objects: 3210, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 3210 (delta 170), reused 162 (delta 86), pack-reused 2940
Receiving objects: 100% (3210/3210), 2.15 MiB | 6.39 MiB/s, done.
Resolving deltas: 100% (2098/2098), done.
Submodule 'rl-trained-agents' (https://github.com/DLR-RM/rl-trained-agents) registered for path 'rl-trained-agents'
Cloning into '/content/rl-baselines3-zoo/rl-trained-agents'...
remote: Enumerating objects: 1706, done.        
remote: Counting objects: 100% (291/291), done.        
remote: Compressing objects: 100% (202/202), done.        
remote: Total 1706 (delta 96), reused 278 (delta 89), pack-reused 1415        
Receiving objects: 100% (1706/1706), 1.17 GiB | 35.57 MiB/s, done.
Resolving deltas: 100% (316/316), done.
Submodule path 'rl-trained-agents': checked out '3dd2af4cee930750016cf943dc6393bada57b89c'
Reading package lists...

In [5]:
import os
from subprocess import Popen, PIPE

import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces
from gym.spaces import Discrete, Box


from stable_baselines3 import DQN, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_vec_env

###Custom Environment de ta te tí

In [7]:
import gym
from gym.spaces import Dict, Discrete, Box, Tuple
import random
import numpy as np

class TicTacToe(gym.Env):
  """
  Ambiente personalizado que sigue la interfaz de gym.
  Es un entorno simple en el cuál el agente debe aprender ganarle a un adversario simple
  """
  metadata = {'render.modes': ['console']}

  def __init__(self, grid_size=10):
    super(TicTacToe, self).__init__()
    
    self.observation_space = Box(low=np.zeros(9), high=np.full((9, ), 3))
    self.state = [0,0,0,0,0,0,0,0,0]
    self.win_combs = [[0,1,2],[3,4,5],[6,7,8], #filas
                       [0,3,6],[1,4,7],[2,5,8], #cols
                       [0,4,8],[2,4,6]] #diags
  
  def step(self, action):
    reward = 0
    pos = 0
    zpos = 0
    #Un valor de action=3 implica poner una Cruz en el tercer lugar vacío (No en el 3er lugar de la grilla, sino el 3ero de los vacíos)
    for st in self.state: 
      if st == 0:
        if zpos == action:
          self.state[pos] = 1
        zpos += 1
      pos += 1  
    if self.check_win():
      reward += 1000
      done = True
    elif self.check_draw():
      reward = reward -100
      done=True
    else:
      self.player2() #Función con IA del adversario
      if self.check_lose():
        reward -= 1000
        done = True
      elif self.check_draw():
        reward = reward -100
        done=True
      else:
        done = False  
    info = {}
    return np.array([self.state]).astype(np.float32), reward, done, info
    


  def player2(self):
    '''Si puede tapar una línea del player 1 la tapa, luego si puede generar una línea la genera, si no, juega random '''
    p2_played = False
    if self.state.count(0) > 0:
      for comb in self.win_combs:
        for skip in [0,1,2]:
          if self.state[comb[skip]]==0 and self.state[comb[[X for X in [0,1,2] if X != skip][0]]]==1 and self.state[comb[[X for X in [0,1,2] if X != skip][1]]]==1:   
            self.state[comb[skip]]=2           
            p2_played = True
            break
        else:
          continue
        break
      
      if not p2_played:
        for comb in self.win_combs:
          for skip in [0,1,2]:
            if self.state[comb[skip]]==0 and self.state[comb[[X for X in [0,1,2] if X != skip][0]]]==2 and self.state[comb[[X for X in [0,1,2] if X != skip][1]]]==2:   
              self.state[comb[skip]]=2           
              p2_played = True
              break
          else:
            continue
          break
        
      if not p2_played:
        random_action = random.randint(0,self.state.count(0)-1)
        pos = 0
        zpos = 0
        for st in self.state:
          if st == 0:
            if zpos == random_action:
              self.state[pos] = 2
            zpos += 1
          pos += 1  
  
  def check_win(self):
    for comb in self.win_combs:
      if self.state[comb[0]]==1 and self.state[comb[1]]==1 and self.state[comb[2]]==1:
        return True
    return False

  def check_lose(self):
    for comb in self.win_combs:
      if self.state[comb[0]]==2 and self.state[comb[1]]==2 and self.state[comb[2]]==2:
        return True
    return False
    
  def check_draw(self):
    if self.state.count(0) > 0:
      return False
    else:
      return True
  
  def reset(self):
    self.state=[0,0,0,0,0,0,0,0,0]

    return np.array([self.state]).astype(np.float32)

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # en nuestra interfaz de consola, representamos el agente como una cruz, y 
    # el resto como un punto
    draw = ["-", "X", "O"]
    drawing = [draw[st] for st in self.state]
    for i in range(0,9,3):
      if i==0:
        print(f"{drawing[i]} {drawing[i+1]} {drawing[i+2]} ")
      if i==3:
        print(f"{drawing[i]} {drawing[i+1]} {drawing[i+2]} ")
      if i==6:
        print(f"{drawing[i]} {drawing[i+1]} {drawing[i+2]} ")
  def close(self):
    pass

  @property
  def action_space(self):
    return Discrete(self.state.count(0))

In [9]:
env = TicTacToe()
env = make_vec_env(lambda: env, n_envs=1)
model = PPO('MlpPolicy', env, verbose=1).learn(500_000)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Se han truncado las últimas 5000 líneas del flujo de salida.
|    value_loss           | 6.63e+05   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.24       |
|    ep_rew_mean          | -656       |
| time/                   |            |
|    fps                  | 1042       |
|    iterations           | 8          |
|    time_elapsed         | 15         |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.00548118 |
|    clip_fraction        | 0.021      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.04      |
|    explained_variance   | 1.11e-05   |
|    learning_rate        | 0.0003     |
|    loss                 | 2.99e+05   |
|    n_updates            | 70         |
|    policy_gradient_loss | -0.0122    |
|    value_loss           | 6.27e+05   |
-------------------------------------

In [10]:
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs)
    print(action)
    obs, rewards, dones, info = env.step(action)
    env.render(mode='console')
    print(dones)
    if dones:
        obs = env.reset()
    print("-----------")

[0]
X - O 
- - - 
- - - 
[False]
-----------
[1]
X - O 
X - - 
O - - 
[False]
-----------
[1]
X - O 
X X O 
O - - 
[False]
-----------
[0]
X X O 
X X O 
O O - 
[False]
-----------
[0]
- - - 
- - - 
- - - 
[ True]
-----------
[0]
X - - 
- O - 
- - - 
[False]
-----------
[1]
X O X 
- O - 
- - - 
[False]
-----------
[0]
X O X 
X O - 
O - - 
[False]
-----------
[0]
X O X 
X O X 
O - O 
[False]
-----------
[0]
- - - 
- - - 
- - - 
[ True]
-----------
[0]
X - - 
- O - 
- - - 
[False]
-----------
[1]
X O X 
- O - 
- - - 
[False]
-----------
[4]
X O X 
- O O 
- - X 
[False]
-----------
[0]
X O X 
X O O 
O - X 
[False]
-----------
[0]
- - - 
- - - 
- - - 
[ True]
-----------
[0]
X - - 
- - - 
O - - 
[False]
-----------
[0]
X X O 
- - - 
O - - 
[False]
-----------
[1]
X X O 
- X - 
O O - 
[False]
-----------
[0]
X X O 
X X O 
O O - 
[False]
-----------
[0]
- - - 
- - - 
- - - 
[ True]
-----------
[0]
X - - 
O - - 
- - - 
[False]
-----------
[1]
X O X 
O - - 
- - - 
[False]
-----------
[0]
X O X 